In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

In [2]:
%%sql
SELECT name,
       type  
  FROM sqlite_master
 WHERE type IN ("table", "view"); 

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [3]:
%%sql 
SELECT *
FROM employee
LIMIT 5;

Done.


employee_id,last_name,first_name,title,reports_to,birthdate,hire_date,address,city,state,country,postal_code,phone,fax,email
1,Adams,Andrew,General Manager,None,1962-02-18 00:00:00,2016-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
2,Edwards,Nancy,Sales Manager,1,1958-12-08 00:00:00,2016-05-01 00:00:00,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com
3,Peacock,Jane,Sales Support Agent,2,1973-08-29 00:00:00,2017-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com
4,Park,Margaret,Sales Support Agent,2,1947-09-19 00:00:00,2017-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
5,Johnson,Steve,Sales Support Agent,2,1965-03-03 00:00:00,2017-10-17 00:00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com


In [4]:
%%sql
SELECT *
FROM customer
LIMIT 5;

Done.


customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
4,Bjørn,Hansen,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4
5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4


In [5]:
%%sql 
WITH usa_tracks_sold AS
    (SELECT il.* 
       FROM invoice_line il
       JOIN invoice i ON il.invoice_id = i.invoice_id 
       JOIN customer c ON i.customer_id = c.customer_id
      WHERE c.country = "USA"
     ) 

SELECT g.name genre,
       COUNT(uts.invoice_line_id) tracks_sold,
       CAST(COUNT(uts.invoice_line_id) AS FLOAT) / (SELECT COUNT(*)
       FROM usa_tracks_sold) percentage_sold
        
  FROM usa_tracks_sold uts 
  JOIN track t ON t.track_id = uts.track_id
  JOIN genre g ON g.genre_id = t.genre_id
 GROUP BY 1
 ORDER BY 2 DESC 
 LIMIT 10;


Done.


genre,tracks_sold,percentage_sold
Rock,561,0.5337773549000951
Alternative & Punk,130,0.12369172216936251
Metal,124,0.11798287345385347
R&B/Soul,53,0.05042816365366318
Blues,36,0.03425309229305423
Alternative,35,0.03330161750713606
Latin,22,0.02093244529019981
Pop,22,0.02093244529019981
Hip Hop/Rap,20,0.019029495718363463
Jazz,14,0.013320647002854425


Rock, Alternative & Punk, and Metal respectively comprise over 75% of tracks sold in the USA. With Rock having 561 tracks sold making up ~53% of all sales. Artists that should be added to the store are Red Tone, Slim Jim Bites, & Meteor and the Girls, respectively.

In [6]:
%%sql
WITH customer_support_rep_sales AS
(
 SELECT i.customer_id, 
        c.support_rep_id,
        SUM(i.total) total
   FROM invoice i
   JOIN customer c ON i.customer_id = c.customer_id
  GROUP BY 1, 2
 )

SELECT e.first_name || " " || e.last_name employee,
       e.hire_date,
       SUM(csrs.total) total_sales
  FROM customer_support_rep_sales csrs
  JOIN employee e ON e.employee_id = csrs.support_rep_id
 GROUP BY 1;
       

Done.


employee,hire_date,total_sales
Jane Peacock,2017-04-01 00:00:00,1731.5099999999998
Margaret Park,2017-05-03 00:00:00,1584.0000000000002
Steve Johnson,2017-10-17 00:00:00,1393.92


While there is a ~20% disparity between the top selling employee and lowest, the  ~5 month hire date difference seems to adequately adjust the rate of sales between employees.

In [7]:
%%sql
WITH country_or_other AS
(
SELECT 
  CASE
  WHEN ( 
       SELECT COUNT(*)
         FROM customer
        WHERE country = c.country
        ) = 1 THEN "Other" 
  ELSE c.country
   END AS country,
       c.customer_id,
       i.*
  FROM invoice i
  JOIN customer c ON c.customer_id = i.customer_id
)

SELECT country,

       customers,
       total_sales,
        average_order,
        value_per_customer
  FROM (
        SELECT country,
               COUNT(distinct customer_id) customers,
               SUM(total) total_sales,
               SUM(total) / COUNT(distinct invoice_id) average_order,
               SUM(total) / COUNT(distinct customer_id) value_per_customer,
               CASE 
                    WHEN country = "Other" THEN 1
                    ELSE 0 
                END AS sort
          FROM country_or_other
         GROUP BY 1
         ORDER BY sort, total_sales DESC
  );
  
  
  
  
       
       

Done.


country,customers,total_sales,average_order,value_per_customer
USA,13,1040.4899999999998,7.942671755725189,80.0376923076923
Canada,8,535.5900000000001,7.047236842105265,66.94875000000002
Brazil,5,427.68000000000006,7.0111475409836075,85.53600000000002
France,5,389.0699999999999,7.781399999999998,77.81399999999998
Germany,4,334.62,8.161463414634147,83.655
Czech Republic,2,273.24000000000007,9.108000000000002,136.62000000000003
United Kingdom,3,245.52,8.768571428571429,81.84
Portugal,2,185.13000000000002,6.383793103448276,92.56500000000001
India,2,183.14999999999998,8.72142857142857,91.57499999999999
Other,15,1094.9400000000003,7.44857142857143,72.99600000000002


According to the results above, there is opportunity to capitalize on in Czech Republic, Portugal, India etc. Due to their low customer amounts coupled with relatively higher total sales and value per customer, there is room to expand the customer base. United Kingdom and India are also viable options due to their large populations and current low customer base. However, due to our relatively small sample size it would be practical to start small on each campaign and adjust the amount of capital allocated based on the results.

## Album vs Individual Tracks

In [15]:
%%sql


WITH invoice_first_track AS

   ( SELECT il.invoice_id invoice_id,
            MIN(il.track_id) first_track_id
       FROM invoice_line il 
      GROUP BY 1
   ) 

SELECT album_purchase,
       COUNT(invoice_id) number_of_invoices,
       CAST(COUNT(invoice_id) AS FLOAT) / 
                                         (SELECT COUNT(*) 
                                            FROM invoice) percent  
        
  
  FROM (SELECT ifs.*,        
          CASE
               WHEN 
                  (
                   SELECT t.track_id FROM track t
                                 
                    WHERE t.album_id = 
                                       
                                      (SELECT t2.album_id FROM track t2
                                        WHERE t2.track_id = ifs.first_track_id)
                      
                   EXCEPT 
                      
                   SELECT il2.track_id FROM invoice_line il2
                    WHERE il2.invoice_id = ifs.invoice_id
                     ) IS NULL
              
                 AND 
        
                   ( 
                   SELECT il2.track_id FROM invoice_line il2
                    WHERE il2.invoice_id = ifs.invoice_id 
                     
                   EXCEPT 
                    
                   SELECT t.track_id FROM track t
                    WHERE t.album_id = 
                                       (SELECT t2.album_id FROM track t2
                                         WHERE t2.track_id = ifs.first_track_id) 
                   )   IS NULL
        
               THEN "yes"
               ELSE "no"
           END AS "album_purchase"
  
  FROM invoice_first_track ifs
       ) 
 

GROUP BY album_purchase;

Done.


album_purchase,number_of_invoices,percent
no,500,0.8143322475570033
yes,114,0.18566775244299674


According to our results, it would not be plausible to stop purchasing albums as they consist of almost 20% of total revenue.